# Backtesting AI vs Rule-based Strategy

This notebook compares the performance of:
- Your AI-driven signals (from `ai_signal_generation.py`)
- Your rule-based signals (e.g., RSI/ATR rules)

**Metrics:**
- Win rate
- Cumulative returns
- Drawdown
- Confusion matrix
- Plots: equity curve, signal comparison


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Load Data
- AI signals from `ai_signals.csv`
- Historical data with features and rule-based signals (from `data_prep.py` output)


In [ ]:
ai_signals = pd.read_csv('ai_signals.csv', index_col=0, parse_dates=True)
# For backtest, reload all historical data
import glob
data_files = glob.glob('data/*.csv')
dfs = [pd.read_csv(f, index_col=0, parse_dates=True) for f in data_files]
df = pd.concat(dfs)
df = df.sort_index()


## Rule-based Signal Example
For demo: Entry if RSI > 60, Exit if RSI < 40. Modify as needed.

In [ ]:
df['rule_signal'] = (df['rsi_14'] > 60).astype(int)


## Merge AI and Rule Signals
Align both signals for comparison.

In [ ]:
df = df.reset_index()
signals = pd.merge(df, ai_signals[['ai_signal','ai_signal_prob']], left_index=True, right_index=True, how='inner')
signals = signals.dropna(subset=['close','rule_signal','ai_signal'])
signals['next_close'] = signals['close'].shift(-1)
signals['return'] = (signals['next_close'] - signals['close']) / signals['close']
signals['ai_return'] = signals['return'] * signals['ai_signal']
signals['rule_return'] = signals['return'] * signals['rule_signal']


## Metrics
- Win rate
- Cumulative returns
- Max drawdown


In [ ]:
def max_drawdown(returns):
    cum = (1 + returns).cumprod()
    peak = cum.cummax()
    dd = (cum - peak) / peak
    return dd.min()

ai_win_rate = (signals['ai_return'] > 0).mean()
rule_win_rate = (signals['rule_return'] > 0).mean()
ai_cum_return = (1 + signals['ai_return']).prod() - 1
rule_cum_return = (1 + signals['rule_return']).prod() - 1
ai_drawdown = max_drawdown(signals['ai_return'])
rule_drawdown = max_drawdown(signals['rule_return'])
print(f'AI Win Rate: {ai_win_rate:.2%}')
print(f'Rule Win Rate: {rule_win_rate:.2%}')
print(f'AI Cumulative Return: {ai_cum_return:.2%}')
print(f'Rule Cumulative Return: {rule_cum_return:.2%}')
print(f'AI Max Drawdown: {ai_drawdown:.2%}')
print(f'Rule Max Drawdown: {rule_drawdown:.2%}')


## Plots
- Equity curves
- Signal comparison


In [ ]:
signals['ai_equity'] = (1 + signals['ai_return']).cumprod()
signals['rule_equity'] = (1 + signals['rule_return']).cumprod()
plt.figure(figsize=(12,6))
plt.plot(signals['ai_equity'], label='AI Equity Curve')
plt.plot(signals['rule_equity'], label='Rule-based Equity Curve')
plt.legend()
plt.title('Equity Curve Comparison')
plt.show()
